In [1]:
import numpy as np
from scipy.fftpack import dct

## First order high pass filter
def high_pass(audio_data, alpha = 0.95):
    return np.append(audio_data[0], audio_data[1:] - alpha * audio_data[:-1])

## Get hamming windowed frames from audio data
def get_frames(audio_data, samplerate, frame_len=25, frame_shift=10):

    window_samples = int(samplerate * frame_len / 1000) # samples
    shift_samples = int(samplerate * frame_shift / 1000) # samples

    num_frames = len(audio_data) // shift_samples
    padded_audio = np.append(audio_data, np.zeros(window_samples - shift_samples))

    frames = np.zeros((num_frames, window_samples))

    hamming = np.hamming(window_samples)

    for i in range(num_frames):
        start = i * shift_samples
        end = start + window_samples
        frames[i] = padded_audio[start:end] * hamming

    return frames

# Get FFT of audio frames
def get_FFT(frames):
    NFFT = 1 << (frames.shape[1] - 1).bit_length() # Gets power of 2 closest to and greater than length of frame
    frames_fft_mag = np.abs(np.fft.rfft(frames, NFFT))

    return frames_fft_mag

## get MFCC from audio frames
def get_MFCC(frames, samplerate, num_mel_filters = 40, num_MFCC=12):
    
    nyquist_freq = samplerate / 2

    # FFT
    frames_fft = get_FFT(frames)

    # Log mel spectrum
    num_filters = num_mel_filters
    fft_len = frames_fft.shape[1]

    low_freq_mel = 0
    high_freq_mel = 1127 * np.log(1 + nyquist_freq / 700)
    mel_points = np.linspace(low_freq_mel, high_freq_mel, num_filters + 2)
    hz_points = 700 * (np.exp(mel_points / 1127) - 1)
    fft_bins = np.floor((hz_points / nyquist_freq) * fft_len)

    mel_filterbank = np.zeros((num_filters, fft_len))
    for m in range(1, num_filters + 1):
        f_m = int(fft_bins[m])
        f_m_left = int(fft_bins[m - 1])
        f_m_right = int(fft_bins[m + 1])

        mel_filterbank[m-1, f_m] = 1
        for k in range(f_m_left, f_m):
            mel_filterbank[m-1, k] = (1 / (f_m - f_m_left)) * (k - f_m) + 1
        for k in range(f_m + 1, f_m_right):
            mel_filterbank[m-1, k] = (1 / (f_m - f_m_right)) * (k - f_m) + 1

    mel_spectrums = np.zeros((len(frames_fft), num_filters))
    for i in range(len(frames_fft)):
        for j in range(len(mel_filterbank)):
            mel_spectrums[i, j] = (frames_fft[i] * mel_filterbank[j]).sum()

    log_mel_spectrums = np.log(mel_spectrums + 1)

    mel_spectrums -= np.mean(mel_spectrums, axis=0) # Mean normalization

    # MFCC 
    MFCC = dct(log_mel_spectrums, type=2, axis=1, norm='ortho')[:, 1 : (num_MFCC + 1)]

    MFCC -= np.mean(MFCC, axis=0) # Mean normalization

    return MFCC

# Uses linear regression over K points around value to get slope
def get_delta(frame_features, K=3):
    num_frames = frame_features.shape[0]
    delta_frame_features = np.zeros(frame_features.shape)

    denominator = 2 * sum([k ** 2 for k in range(1, K+1)])
    for i in range(num_frames):
        # j is adusted index at boundaries 
        if i < K:
            j = K
        elif i + K > (num_frames - 1):
            j = (num_frames - 1 - K)
        else:
            j = i

        numerator = sum([k * frame_features[j + k] - frame_features[j - k] for k in range(1, K + 1)])

        delta_frame_features[i] = numerator / denominator
    
    return delta_frame_features

In [2]:
from scipy.io import wavfile

samplerate, data = wavfile.read("DARPA Audio Data/data/TRAIN/DR1/FCJF0/SA1.WAV.wav")

high_pass_audio = high_pass(data)
frames = get_frames(high_pass_audio, samplerate)
frame_energy = (frames ** 2).sum(axis=1).reshape(-1, 1)
frame_MFCC = get_MFCC(frames, samplerate)
delta_frame_energy = get_delta(frame_energy)
delta_frame_MFCC = get_delta(frame_MFCC)
double_delte_frame_energy = get_delta(delta_frame_energy)
double_delta_frame_MFCC = get_delta(delta_frame_MFCC)

all_frame_features = np.concatenate([frame_MFCC, delta_frame_MFCC, double_delta_frame_MFCC, frame_energy, delta_frame_energy, double_delte_frame_energy], axis=1)
